## Basics

- Installations: covalent, pytorch, torchvision
- Import libraries
- Load dataset
- Clean dataset (?)

## Workflow without Covalent

- Build network
- Train model
- Evaluate model performance
- Time performance
- Results

## Workflow with Covalent

- Covert workflow outline previously to electrons and lattices
- Evaluate time performance
- Double check model performance
- Results

## Comparison and discussion of Covalent concepts

In [1]:
# !pip install cova
# !conda install pytorch torchvision -c pytorch -y

In [2]:
import os
from pathlib import Path

import torch
import torchvision

In [3]:
n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [4]:
def data_loader(batch_size: int,
                train: bool,
                download: bool = True,
                shuffle: bool = True,
                data_dir: str = '~/data/mnist/',
                alpha: float = 0.1307,  # normalization parameter
                beta: float = 0.3081  # normalization parameter
                ) -> torch.utils.data.dataloader.DataLoader:
    """Data loader"""

    data_dir = Path(data_dir).expanduser()  # Get result director absolute path
    data_dir.mkdir(parents=True, exist_ok=True)  # Create directories if they don't exist    

    return torch.utils.data.DataLoader(
        torchvision.datasets.MNIST(
            data_dir,
            train=train,
            download=download,
            transform=torchvision.transforms.Compose([
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((alpha,), (beta,))
            ])),
        batch_size=batch_size,
        shuffle=shuffle)


train_loader = data_loader(batch_size=batch_size_train, train=True)
test_loader = data_loader(batch_size=batch_size_test, train=False)

In [5]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [7]:
network = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)

In [8]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i * len(train_loader.dataset) for i in range(n_epochs + 1)]

def train(epoch: int, results_dir: str = '~/data/mnist/results/') -> None:
    results_dir = Path(results_dir).expanduser()  # Get result director absolute path
    results_dir.mkdir(parents=True, exist_ok=True)  # Create directories if they don't exist

    network.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = network(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')
            train_losses.append(loss.item())
            train_counter.append(
                (batch_idx * 64) + ((epoch - 1) * len(train_loader.dataset)))
            torch.save(network.state_dict(), f'{results_dir}model.pth')
            torch.save(optimizer.state_dict(), f'{results_dir}optimizer.pth')

In [9]:
def test():
    network.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = network(data)
            test_loss += F.nll_loss(output, target, size_average=False).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)
    print(f'\nTest set: Avg. loss: {test_loss:.4f}, Accuracy: {correct}/'
          f'{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n')

In [10]:
test()
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()

/var/folders/hn/dnjxkq7946j9wpxv1cy9pm2m0000gn/T/ipykernel_25195/161431047.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
/Users/faiyaz/opt/anaconda3/envs/covalent-pytorch-dev/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 2.3316, Accuracy: 1137/10000 (11%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.360446
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.364010
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.329987
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.249067
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.288059
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.218085
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.214215
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.111192
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.092593
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.989863
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.911151
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.659503
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.869479
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.660094
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.577335
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.550470
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.304469
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.408615
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.306857
Train Epoch: 1 [12160

Triple check to ensure there is no plagiarism issues
References:
- https://nextjournal.com/gkoehler/pytorch-mnist
- https://towardsdatascience.com/handwritten-digit-mnist-pytorch-977b5338e627